In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
# import os
# import sys
# notebook_path = os.path.abspath('')

# # Find the part of the path that contains 'commitFit'
# commit_fit_path = None
# for part in notebook_path.split(os.sep):
#     print(part)
#     if 'CommitFit' in part:
#         commit_fit_path = notebook_path.split(part)[0] + part
#         break

# if commit_fit_path is None:
#     raise ValueError("Path containing 'commitFit' not found in notebook path.")

# # Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
# if commit_fit_path not in sys.path:
#     sys.path.append(commit_fit_path)
# from evaluate.metrics import accuracy,precision,recall,f1
# CUDA_LAUNCH_BLOCKING=1
import evaluate

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_6775/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_6775/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,0,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,0,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,1,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,0,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,0,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,0,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,0,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,0,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,1,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [5]:
len(train)

101

In [6]:
train['label'].value_counts()

label
0    63
1    38
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    6284
1    3727
Name: count, dtype: int64

In [8]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [9]:
train

,label,text,diff
0,0,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,0,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,1,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,0,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,0,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,0,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,0,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,0,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,1,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [10]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "../../bert-base-cased", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained('../../bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [14]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [16]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('../../evaluate/metrics/accuracy')
    precision =  evaluate.load("../../evaluate/metrics/precision")
    recall =  evaluate.load("../../evaluate/metrics/recall")
    f1 =  evaluate.load("../../evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [17]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [18]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> model:
    # params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return model
def my_compute_objective(metrics):
    print('+++++++++++',metrics['eval_accuracy']['accuracy'])
    return  metrics['eval_accuracy']['accuracy']

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    model_init= model_init,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:423: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2024-05-29 17:53:20,429] A new study created in memory with name: no-name-41775913-15ef-4318-b6c4-9d212715fa33
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_6775/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.651520,{'precision': 0.3940192409067237},{'recall': 0.6277095195285186},{'f1': 0.4841395054577736},{'accuracy': 0.6277095195285186}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


+++++++++++ 0.6277095195285186


[I 2024-05-29 17:53:58,377] Trial 0 finished with value: 0.6277095195285186 and parameters: {'learning_rate': 6.065984487299014e-05}. Best is trial 0 with value: 0.6277095195285186.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_6775/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


...

[I 2024-05-29 17:58:54,929] Trial 8 pruned. 
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_6775/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.660574,{'precision': 0.3940192409067237},{'recall': 0.6277095195285186},{'f1': 0.4841395054577736},{'accuracy': 0.6277095195285186}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


+++++++++++ 0.6277095195285186


[I 2024-05-29 17:59:37,272] Trial 9 pruned. 


In [20]:
best_run

BestRun(run_id='7', objective=0.6514833682948756, hyperparameters={'learning_rate': 3.640545020194478e-06}, run_summary=None)